# Big Data Principal Component Analysis (PCA) Implementation
## CSE 4460 - Big Data Analytics Lab - Open Ended Experiment

### Project Overview
This notebook demonstrates a scalable PCA implementation using Apache Spark for high-dimensional datasets. We'll work with an expanded California Housing dataset containing millions of records with engineered features that create high dimensionality and redundancy.

### Learning Objectives
- Understand big data architecture for PCA
- Implement distributed PCA using Apache Spark
- Handle high-dimensional data with memory constraints
- Validate and visualize PCA results
- Analyze performance and scalability trade-offs

## 1. Environment Setup and Dependencies

First, let's install and import all necessary libraries for our big data PCA analysis.

In [ ]:
# Install required packages
!pip install pyspark==3.5.1
!pip install pandas numpy matplotlib seaborn scikit-learn
!pip install plotly

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# PySpark imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler as SparkStandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.mllib.linalg.distributed import RowMatrix

# Set display options
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ All libraries imported successfully!")

## 2. Big Data Architecture Setup

### 2.1 Spark Session Configuration
We'll configure Spark for optimal PCA performance with memory management and parallelization settings.

In [ ]:
# Configure Spark Session for PCA workload
def create_spark_session():
    """
    Create optimized Spark session for PCA operations
    - Increased driver memory for large matrices
    - Optimized serialization for ML operations
    - Configured for local cluster simulation
    """
    spark = SparkSession.builder \
        .appName("BigData_PCA_Analysis") \
        .config("spark.driver.memory", "4g") \
        .config("spark.driver.maxResultSize", "2g") \
        .config("spark.executor.memory", "2g") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .master("local[*]") \
        .getOrCreate()
    
    # Set log level to reduce noise
    spark.sparkContext.setLogLevel("WARN")
    
    return spark

# Create Spark session
spark = create_spark_session()

print(f"✅ Spark Session Created!")
print(f"Spark Version: {spark.version}")
print(f"Available Cores: {spark.sparkContext.defaultParallelism}")
print(f"Driver Memory: {spark.conf.get('spark.driver.memory')}")

### 2.2 Big Data Architecture Overview

**Architecture Components:**
1. **Data Storage Layer**: Distributed file system (simulated with local partitions)
2. **Processing Layer**: Apache Spark with MLlib for distributed linear algebra
3. **Memory Management**: Optimized for large matrix operations
4. **Compute Distribution**: Parallel processing across multiple cores

**PCA-Specific Challenges:**
- **Memory Constraints**: Covariance matrix computation requires O(d²) memory
- **Computational Complexity**: SVD computation is O(nd²) where n=samples, d=dimensions
- **Data Distribution**: Need to minimize data shuffling during matrix operations
- **Scalability**: Must handle datasets that don't fit in single-machine memory

## 3. High-Dimensional Dataset Creation

We'll create a large, high-dimensional dataset based on California Housing data with engineered features that introduce redundancy and multicollinearity - perfect conditions for PCA.

In [ ]:
def create_high_dimensional_dataset(n_samples=1000000, n_features_base=8, expansion_factor=15):
    """
    Create a large, high-dimensional dataset with redundancy and multicollinearity
    
    Parameters:
    - n_samples: Target number of samples (default: 1M)
    - n_features_base: Base features from California housing
    - expansion_factor: Factor to expand feature space
    
    Returns:
    - pandas DataFrame with high-dimensional features
    """
    print("🔄 Creating high-dimensional dataset...")
    
    # Load base California housing data
    california_housing = fetch_california_housing()
    base_data = california_housing.data
    feature_names = california_housing.feature_names
    target = california_housing.target
    
    # Replicate data to reach target sample size
    replications = n_samples // len(base_data) + 1
    
    # Replicate and add noise to create variations
    expanded_data = []
    expanded_targets = []
    
    np.random.seed(42)  # For reproducibility
    
    for i in range(replications):
        # Add different noise patterns for each replication
        noise_scale = 0.1 + (i * 0.05)  # Increasing noise with replications
        noisy_data = base_data + np.random.normal(0, noise_scale, base_data.shape)
        noisy_target = target + np.random.normal(0, 0.5, len(target))
        
        expanded_data.append(noisy_data)
        expanded_targets.append(noisy_target)
    
    # Combine all replications
    final_data = np.vstack(expanded_data)[:n_samples]
    final_targets = np.hstack(expanded_targets)[:n_samples]
    
    # Create DataFrame with base features
    df = pd.DataFrame(final_data, columns=feature_names)
    df['target'] = final_targets
    
    # Add high-dimensional engineered features that create redundancy
    print("🔄 Engineering high-dimensional features...")
    
    # 1. Polynomial features (create multicollinearity)
    df['MedInc_squared'] = df['MedInc'] ** 2
    df['HouseAge_squared'] = df['HouseAge'] ** 2
    df['AveRooms_squared'] = df['AveRooms'] ** 2
    
    # 2. Interaction features (more multicollinearity)
    df['MedInc_x_HouseAge'] = df['MedInc'] * df['HouseAge']
    df['AveRooms_x_AveBedrms'] = df['AveRooms'] * df['AveBedrms']
    df['Population_x_AveOccup'] = df['Population'] * df['AveOccup']
    
    # 3. Logarithmic transformations
    df['log_MedInc'] = np.log(df['MedInc'] + 1)
    df['log_Population'] = np.log(df['Population'] + 1)
    df['log_AveRooms'] = np.log(df['AveRooms'] + 1)
    
    # 4. Ratio features
    df['Rooms_per_Household'] = df['AveRooms'] / (df['AveOccup'] + 0.001)
    df['Bedrooms_per_Room'] = df['AveBedrms'] / (df['AveRooms'] + 0.001)
    df['Population_density'] = df['Population'] / (df['AveOccup'] + 0.001)
    
    # 5. Binned features (categorical-like)
    df['MedInc_bin_low'] = (df['MedInc'] < df['MedInc'].quantile(0.33)).astype(int)
    df['MedInc_bin_mid'] = ((df['MedInc'] >= df['MedInc'].quantile(0.33)) & 
                           (df['MedInc'] < df['MedInc'].quantile(0.67))).astype(int)
    df['MedInc_bin_high'] = (df['MedInc'] >= df['MedInc'].quantile(0.67)).astype(int)
    
    # 6. Geographic features (based on Latitude/Longitude)
    df['Lat_rounded'] = np.round(df['Latitude'], 1)
    df['Lon_rounded'] = np.round(df['Longitude'], 1)
    df['Geographic_cluster'] = df['Lat_rounded'] * 100 + df['Lon_rounded']
    
    # 7. Time-based synthetic features (simulate time series)
    time_periods = np.random.choice(range(1, 13), size=len(df))  # Months 1-12
    df['Month'] = time_periods
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    
    # 8. Add more synthetic correlated features
    for i in range(expansion_factor):
        # Create features that are linear combinations of existing features
        weights = np.random.normal(0, 0.5, len(feature_names))
        synthetic_feature = np.dot(final_data, weights) + np.random.normal(0, 0.1, n_samples)
        df[f'synthetic_feature_{i}'] = synthetic_feature
        
        # Add noisy versions of existing features
        base_feature = np.random.choice(feature_names)
        df[f'noisy_{base_feature}_{i}'] = (df[base_feature] + 
                                          np.random.normal(0, 0.2 * df[base_feature].std(), n_samples))
    
    print(f"✅ Dataset created with {len(df)} samples and {len(df.columns)} features")
    print(f"📊 Dataset size: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return df

# Create the high-dimensional dataset
start_time = time.time()
df_large = create_high_dimensional_dataset(n_samples=500000, expansion_factor=10)  # 500K samples for demo
creation_time = time.time() - start_time

print(f"⏱️  Dataset creation time: {creation_time:.2f} seconds")
print(f"📈 Final dataset shape: {df_large.shape}")

### 3.1 Dataset Analysis and Validation

In [ ]:
# Analyze the created dataset
print("📊 Dataset Overview:")
print(f"Shape: {df_large.shape}")
print(f"Memory usage: {df_large.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Data types: {df_large.dtypes.value_counts().to_dict()}")

# Check for multicollinearity (correlation matrix sample)
print("\n🔍 Checking for Multicollinearity (first 15 features):")
correlation_matrix = df_large.iloc[:, :15].corr()

# Find highly correlated feature pairs
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

print(f"Found {len(high_corr_pairs)} highly correlated feature pairs (|r| > 0.7)")
for pair in high_corr_pairs[:5]:  # Show first 5
    print(f"  {pair[0]} ↔ {pair[1]}: r = {pair[2]:.3f}")

# Basic statistics
print("\n📈 Basic Statistics:")
print(df_large.describe())

## 4. Data Loading into Spark

Now we'll load our high-dimensional dataset into Spark DataFrame for distributed processing.

In [ ]:
def load_data_to_spark(pandas_df, spark_session, partition_count=None):
    """
    Load pandas DataFrame to Spark with optimal partitioning
    
    Parameters:
    - pandas_df: Input pandas DataFrame
    - spark_session: Active Spark session
    - partition_count: Number of partitions (auto if None)
    
    Returns:
    - Spark DataFrame
    """
    print("🔄 Loading data into Spark DataFrame...")
    
    # Convert to Spark DataFrame
    spark_df = spark_session.createDataFrame(pandas_df)
    
    # Optimize partitioning for PCA operations
    if partition_count is None:
        # Rule of thumb: aim for 128MB per partition
        data_size_mb = pandas_df.memory_usage(deep=True).sum() / 1024**2
        partition_count = max(int(data_size_mb / 128), spark_session.sparkContext.defaultParallelism)
    
    spark_df = spark_df.repartition(partition_count)
    
    # Cache for multiple operations
    spark_df.cache()
    
    # Trigger caching
    row_count = spark_df.count()
    
    print(f"✅ Data loaded into Spark DataFrame")
    print(f"📊 Rows: {row_count:,}")
    print(f"📊 Columns: {len(spark_df.columns)}")
    print(f"🔧 Partitions: {spark_df.rdd.getNumPartitions()}")
    
    return spark_df

# Load data to Spark
start_time = time.time()
spark_df = load_data_to_spark(df_large, spark)
loading_time = time.time() - start_time

print(f"⏱️  Data loading time: {loading_time:.2f} seconds")

# Show schema
print("\n📋 Spark DataFrame Schema:")
spark_df.printSchema()

## 5. Data Preprocessing Pipeline

Before applying PCA, we need to preprocess the data:
1. Handle missing values
2. Feature selection
3. Feature scaling
4. Vector assembly for MLlib

In [ ]:
def preprocess_for_pca(spark_df, target_column='target'):
    """
    Comprehensive preprocessing pipeline for PCA
    
    Steps:
    1. Remove target column and non-numeric features
    2. Handle missing values
    3. Feature selection (remove low-variance features)
    4. Assemble features into vector
    5. Scale features
    
    Parameters:
    - spark_df: Input Spark DataFrame
    - target_column: Name of target column to exclude
    
    Returns:
    - processed_df: DataFrame ready for PCA
    - feature_columns: List of feature column names
    - scaler_model: Fitted StandardScaler model
    """
    print("🔄 Starting data preprocessing pipeline...")
    
    # 1. Select numeric features (exclude target)
    numeric_columns = []
    for field in spark_df.schema.fields:
        if (field.dataType in [DoubleType(), FloatType(), IntegerType(), LongType()] and 
            field.name != target_column):
            numeric_columns.append(field.name)
    
    print(f"📊 Selected {len(numeric_columns)} numeric features for PCA")
    
    # 2. Select only numeric columns
    df_numeric = spark_df.select(numeric_columns)
    
    # 3. Handle missing values (fill with column mean)
    print("🔧 Handling missing values...")
    
    # Calculate means for missing value imputation
    means = {}
    for col in numeric_columns:
        mean_val = df_numeric.agg(F.mean(col)).collect()[0][0]
        if mean_val is not None:
            means[col] = float(mean_val)
        else:
            means[col] = 0.0
    
    # Fill missing values
    df_clean = df_numeric.fillna(means)
    
    # 4. Feature variance analysis (remove low-variance features)
    print("🔧 Analyzing feature variance...")
    
    variances = {}
    for col in numeric_columns:
        var_val = df_clean.agg(F.variance(col)).collect()[0][0]
        if var_val is not None:
            variances[col] = float(var_val)
        else:
            variances[col] = 0.0
    
    # Remove features with very low variance (threshold: 1e-6)
    low_variance_threshold = 1e-6
    high_variance_features = [col for col, var in variances.items() if var > low_variance_threshold]
    
    print(f"🔧 Removed {len(numeric_columns) - len(high_variance_features)} low-variance features")
    print(f"📊 Final feature count: {len(high_variance_features)}")
    
    # 5. Select final features
    df_selected = df_clean.select(high_variance_features)
    
    # 6. Assemble features into vector column
    print("🔧 Assembling feature vectors...")
    
    vector_assembler = VectorAssembler(
        inputCols=high_variance_features,
        outputCol="features_raw"
    )
    
    df_assembled = vector_assembler.transform(df_selected)
    
    # 7. Scale features
    print("🔧 Scaling features...")
    
    scaler = SparkStandardScaler(
        inputCol="features_raw",
        outputCol="features_scaled",
        withStd=True,
        withMean=True
    )
    
    scaler_model = scaler.fit(df_assembled)
    df_scaled = scaler_model.transform(df_assembled)
    
    # 8. Final selection
    final_df = df_scaled.select("features_scaled").withColumnRenamed("features_scaled", "features")
    
    # Cache the result
    final_df.cache()
    final_row_count = final_df.count()  # Trigger caching
    
    print(f"✅ Preprocessing completed!")
    print(f"📊 Final dataset: {final_row_count:,} rows × {len(high_variance_features)} features")
    
    return final_df, high_variance_features, scaler_model

# Apply preprocessing pipeline
start_time = time.time()
processed_df, feature_names, scaler_model = preprocess_for_pca(spark_df)
preprocessing_time = time.time() - start_time

print(f"⏱️  Preprocessing time: {preprocessing_time:.2f} seconds")
print(f"🎯 Ready for PCA with {len(feature_names)} features")

## 6. Distributed PCA Implementation

Now we'll implement PCA using Spark's MLlib for distributed linear algebra operations.

In [ ]:
def distributed_pca_analysis(spark_df, n_components=None, explained_variance_threshold=0.95):
    """
    Perform distributed PCA using Spark MLlib
    
    Parameters:
    - spark_df: Preprocessed Spark DataFrame with 'features' column
    - n_components: Number of components (None for auto-selection)
    - explained_variance_threshold: Threshold for automatic component selection
    
    Returns:
    - Dictionary containing PCA results and analysis
    """
    print("🔄 Starting distributed PCA analysis...")
    
    # Convert Spark DataFrame to RDD of MLlib Vectors
    print("🔧 Converting to RDD format for MLlib...")
    
    def extract_features(row):
        return MLLibVectors.dense(row.features.toArray())
    
    features_rdd = spark_df.rdd.map(extract_features)
    
    # Create RowMatrix for distributed linear algebra
    print("🔧 Creating RowMatrix for distributed operations...")
    row_matrix = RowMatrix(features_rdd)
    
    print(f"📊 Matrix dimensions: {row_matrix.numRows()} × {row_matrix.numCols()}")
    
    # Compute SVD (Singular Value Decomposition)
    print("🔧 Computing SVD (this may take several minutes for large datasets)...")
    
    start_svd = time.time()
    
    # For large datasets, we limit the number of components to compute
    max_components = min(row_matrix.numCols(), 100)  # Limit to 100 components for demo
    
    if n_components is None:
        n_components = max_components
    else:
        n_components = min(n_components, max_components)
    
    # Compute SVD
    svd = row_matrix.computeSVD(n_components, computeU=True)
    
    svd_time = time.time() - start_svd
    print(f"⏱️  SVD computation time: {svd_time:.2f} seconds")
    
    # Extract components
    U = svd.U  # Left singular vectors (not needed for PCA)
    s = svd.s  # Singular values
    V = svd.V  # Right singular vectors (principal components)
    
    # Convert singular values to numpy for analysis
    singular_values = np.array(s)
    
    # Compute explained variance
    print("📊 Computing explained variance...")
    
    # Variance is proportional to squared singular values
    explained_variance = (singular_values ** 2) / (row_matrix.numRows() - 1)
    total_variance = np.sum(explained_variance)
    explained_variance_ratio = explained_variance / total_variance
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
    
    # Find optimal number of components
    optimal_components = np.argmax(cumulative_variance_ratio >= explained_variance_threshold) + 1
    
    # Extract principal components matrix
    components_matrix = V.toArray().T  # Transpose to get components as rows
    
    print(f"✅ PCA Analysis completed!")
    print(f"📊 Components computed: {len(singular_values)}")
    print(f"📊 Optimal components (for {explained_variance_threshold*100}% variance): {optimal_components}")
    print(f"📊 Variance explained by first component: {explained_variance_ratio[0]*100:.2f}%")
    print(f"📊 Cumulative variance (first 5 components): {cumulative_variance_ratio[4]*100:.2f}%")
    
    # Package results
    results = {
        'n_components': len(singular_values),
        'optimal_components': optimal_components,
        'singular_values': singular_values,
        'explained_variance': explained_variance,
        'explained_variance_ratio': explained_variance_ratio,
        'cumulative_variance_ratio': cumulative_variance_ratio,
        'components': components_matrix,
        'feature_names': feature_names,
        'svd_time': svd_time,
        'total_variance': total_variance
    }
    
    return results

# Perform distributed PCA
print("🚀 Starting distributed PCA computation...")
start_time = time.time()

pca_results = distributed_pca_analysis(
    processed_df, 
    n_components=50,  # Limit for demonstration
    explained_variance_threshold=0.95
)

total_pca_time = time.time() - start_time
print(f"⏱️  Total PCA analysis time: {total_pca_time:.2f} seconds")

## 7. PCA Results Validation and Comparison

Let's validate our distributed PCA results by comparing with scikit-learn's PCA on a sample of the data.

In [ ]:
def validate_pca_results(pca_results, processed_df, sample_size=10000):
    """
    Validate distributed PCA results against scikit-learn PCA
    
    Parameters:
    - pca_results: Results from distributed PCA
    - processed_df: Preprocessed Spark DataFrame
    - sample_size: Size of sample for scikit-learn comparison
    
    Returns:
    - Validation metrics and comparison
    """
    print("🔄 Validating PCA results against scikit-learn...")
    
    # Sample data for scikit-learn comparison
    print(f"📊 Sampling {sample_size} rows for validation...")
    
    sample_df = processed_df.sample(False, sample_size / processed_df.count()).limit(sample_size)
    
    # Convert to pandas for scikit-learn
    sample_features = np.array([row.features.toArray() for row in sample_df.collect()])
    
    print(f"📊 Sample shape: {sample_features.shape}")
    
    # Apply scikit-learn PCA
    print("🔧 Running scikit-learn PCA for comparison...")
    
    start_sklearn = time.time()
    sklearn_pca = PCA(n_components=min(pca_results['n_components'], sample_features.shape[1]))
    sklearn_pca.fit(sample_features)
    sklearn_time = time.time() - start_sklearn
    
    print(f"⏱️  Scikit-learn PCA time: {sklearn_time:.2f} seconds")
    
    # Compare explained variance ratios
    spark_explained_var = pca_results['explained_variance_ratio'][:len(sklearn_pca.explained_variance_ratio_)]
    sklearn_explained_var = sklearn_pca.explained_variance_ratio_
    
    # Calculate comparison metrics
    variance_diff = np.abs(spark_explained_var - sklearn_explained_var)
    mean_variance_diff = np.mean(variance_diff)
    max_variance_diff = np.max(variance_diff)
    
    print("📊 Validation Results:")
    print(f"  Mean explained variance difference: {mean_variance_diff:.6f}")
    print(f"  Max explained variance difference: {max_variance_diff:.6f}")
    print(f"  Correlation of explained variances: {np.corrcoef(spark_explained_var, sklearn_explained_var)[0,1]:.6f}")
    
    # Performance comparison
    print("⚡ Performance Comparison:")
    print(f"  Distributed PCA time: {pca_results['svd_time']:.2f} seconds")
    print(f"  Scikit-learn PCA time: {sklearn_time:.2f} seconds")
    print(f"  Data size ratio: {processed_df.count() / sample_size:.1f}x")
    
    validation_results = {
        'mean_variance_diff': mean_variance_diff,
        'max_variance_diff': max_variance_diff,
        'variance_correlation': np.corrcoef(spark_explained_var, sklearn_explained_var)[0,1],
        'sklearn_explained_variance': sklearn_explained_var,
        'spark_explained_variance': spark_explained_var,
        'sklearn_time': sklearn_time,
        'spark_time': pca_results['svd_time'],
        'sample_size': sample_size
    }
    
    return validation_results

# Validate results
validation_results = validate_pca_results(pca_results, processed_df, sample_size=5000)

if validation_results['mean_variance_diff'] < 0.01:
    print("✅ Validation PASSED: Results are consistent with scikit-learn")
else:
    print("⚠️  Validation WARNING: Some differences detected (expected due to sampling)")

## 8. Visualization and Interpretation

Let's create comprehensive visualizations to understand our PCA results.

In [ ]:
def create_pca_visualizations(pca_results, validation_results):
    """
    Create comprehensive PCA visualizations
    
    Parameters:
    - pca_results: Results from distributed PCA
    - validation_results: Validation comparison results
    """
    print("🎨 Creating PCA visualizations...")
    
    # Set up the plotting environment
    plt.rcParams['figure.figsize'] = [15, 12]
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Big Data PCA Analysis Results', fontsize=16, fontweight='bold')
    
    # 1. Explained Variance Ratio
    ax1 = axes[0, 0]
    components_range = range(1, len(pca_results['explained_variance_ratio']) + 1)
    ax1.bar(components_range[:20], pca_results['explained_variance_ratio'][:20], 
            color='skyblue', alpha=0.7, edgecolor='navy')
    ax1.set_xlabel('Principal Component')
    ax1.set_ylabel('Explained Variance Ratio')
    ax1.set_title('Explained Variance by Component\n(First 20 Components)')
    ax1.grid(True, alpha=0.3)
    
    # 2. Cumulative Explained Variance
    ax2 = axes[0, 1]
    ax2.plot(components_range, pca_results['cumulative_variance_ratio'], 
             'bo-', linewidth=2, markersize=4)
    ax2.axhline(y=0.95, color='red', linestyle='--', 
                label=f'95% Threshold (Component {pca_results["optimal_components"]})')
    ax2.axhline(y=0.90, color='orange', linestyle='--', alpha=0.7, label='90% Threshold')
    ax2.set_xlabel('Number of Components')
    ax2.set_ylabel('Cumulative Explained Variance')
    ax2.set_title('Cumulative Explained Variance')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Singular Values (Scree Plot)
    ax3 = axes[0, 2]
    ax3.plot(components_range, pca_results['singular_values'], 'go-', linewidth=2, markersize=4)
    ax3.set_xlabel('Component Number')
    ax3.set_ylabel('Singular Value')
    ax3.set_title('Scree Plot (Singular Values)')
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
    
    # 4. Feature Contributions to PC1
    ax4 = axes[1, 0]
    pc1_contributions = np.abs(pca_results['components'][0])
    top_features_idx = np.argsort(pc1_contributions)[-15:]  # Top 15 features
    top_features = [pca_results['feature_names'][i] for i in top_features_idx]
    top_contributions = pc1_contributions[top_features_idx]
    
    y_pos = np.arange(len(top_features))
    ax4.barh(y_pos, top_contributions, color='lightcoral', alpha=0.8)
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels([name[:15] + '...' if len(name) > 15 else name for name in top_features], 
                        fontsize=8)
    ax4.set_xlabel('Absolute Contribution')
    ax4.set_title('Top Feature Contributions to PC1')
    ax4.grid(True, alpha=0.3, axis='x')
    
    # 5. Validation Comparison
    ax5 = axes[1, 1]
    comparison_components = range(1, len(validation_results['spark_explained_variance']) + 1)
    ax5.plot(comparison_components, validation_results['spark_explained_variance'], 
             'bo-', label='Distributed PCA (Spark)', linewidth=2, markersize=4)
    ax5.plot(comparison_components, validation_results['sklearn_explained_variance'], 
             'ro-', label='Scikit-learn PCA', linewidth=2, markersize=4, alpha=0.7)
    ax5.set_xlabel('Component Number')
    ax5.set_ylabel('Explained Variance Ratio')
    ax5.set_title('PCA Validation: Spark vs Scikit-learn')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Performance Metrics
    ax6 = axes[1, 2]
    metrics = ['Mean Var Diff', 'Max Var Diff', 'Correlation']
    values = [validation_results['mean_variance_diff'], 
              validation_results['max_variance_diff'],
              validation_results['variance_correlation']]
    colors = ['green' if v < 0.01 else 'orange' if v < 0.05 else 'red' for v in values[:-1]] + ['green']
    
    bars = ax6.bar(metrics, values, color=colors, alpha=0.7)
    ax6.set_ylabel('Value')
    ax6.set_title('Validation Metrics')
    ax6.set_ylim(0, max(max(values), 1.0))
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{value:.4f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Additional summary statistics
    print("\n📊 PCA Analysis Summary:")
    print(f"  Total features analyzed: {len(pca_results['feature_names'])}")
    print(f"  Principal components computed: {pca_results['n_components']}")
    print(f"  Components for 95% variance: {pca_results['optimal_components']}")
    print(f"  Dimensionality reduction ratio: {pca_results['optimal_components'] / len(pca_results['feature_names']):.2%}")
    print(f"  First component explains: {pca_results['explained_variance_ratio'][0]*100:.2f}% of variance")
    print(f"  Top 5 components explain: {pca_results['cumulative_variance_ratio'][4]*100:.2f}% of variance")

# Create visualizations
create_pca_visualizations(pca_results, validation_results)

## 9. Performance Analysis and Scalability

Let's analyze the performance characteristics and scalability of our distributed PCA implementation.

In [ ]:
def analyze_performance_scalability():
    """
    Analyze performance characteristics and provide scalability insights
    """
    print("📈 Performance Analysis and Scalability Assessment")
    print("=" * 60)
    
    # Collect timing information
    timings = {
        'Dataset Creation': creation_time,
        'Data Loading to Spark': loading_time,
        'Preprocessing Pipeline': preprocessing_time,
        'SVD Computation': pca_results['svd_time'],
        'Total PCA Analysis': total_pca_time
    }
    
    # Dataset characteristics
    dataset_stats = {
        'Samples': processed_df.count(),
        'Original Features': len(df_large.columns),
        'Final Features': len(feature_names),
        'Data Size (MB)': df_large.memory_usage(deep=True).sum() / 1024**2,
        'Spark Partitions': processed_df.rdd.getNumPartitions(),
        'Available Cores': spark.sparkContext.defaultParallelism
    }
    
    print("🔢 Dataset Statistics:")
    for key, value in dataset_stats.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value:,}")
    
    print("\n⏱️  Performance Breakdown:")
    total_time = sum(timings.values())
    for operation, time_taken in timings.items():
        percentage = (time_taken / total_time) * 100
        print(f"  {operation}: {time_taken:.2f}s ({percentage:.1f}%)")
    
    print(f"\n🎯 Total Processing Time: {total_time:.2f} seconds")
    
    # Calculate throughput metrics
    samples_per_second = dataset_stats['Samples'] / total_time
    features_per_second = dataset_stats['Final Features'] * dataset_stats['Samples'] / total_time
    
    print(f"\n📊 Throughput Metrics:")
    print(f"  Samples processed per second: {samples_per_second:,.0f}")
    print(f"  Feature-samples processed per second: {features_per_second:,.0f}")
    
    # Memory efficiency analysis
    memory_per_sample = (dataset_stats['Data Size (MB)'] * 1024 * 1024) / dataset_stats['Samples']
    print(f"\n💾 Memory Efficiency:")
    print(f"  Memory per sample: {memory_per_sample:.0f} bytes")
    print(f"  Data distributed across: {dataset_stats['Spark Partitions']} partitions")
    print(f"  Approximate partition size: {dataset_stats['Data Size (MB)'] / dataset_stats['Spark Partitions']:.2f} MB")
    
    # Scalability projections
    print(f"\n🚀 Scalability Projections:")
    
    # Project performance for different dataset sizes
    current_samples = dataset_stats['Samples']
    current_features = dataset_stats['Final Features']
    
    projections = [
        ('1M samples', 1_000_000, current_features),
        ('10M samples', 10_000_000, current_features),
        ('1M samples, 200 features', 1_000_000, 200),
        ('10M samples, 500 features', 10_000_000, 500)
    ]
    
    for scenario, proj_samples, proj_features in projections:
        # SVD complexity is roughly O(min(n*d², d³)) where n=samples, d=features
        complexity_ratio = (proj_samples / current_samples) * (proj_features / current_features) ** 2
        projected_time = pca_results['svd_time'] * complexity_ratio
        
        print(f"  {scenario}: ~{projected_time:.1f}s (PCA computation)")
    
    # Recommendations
    print(f"\n💡 Optimization Recommendations:")
    
    if pca_results['svd_time'] / total_time > 0.7:
        print("  • SVD computation dominates runtime - consider incremental PCA for larger datasets")
    
    if dataset_stats['Spark Partitions'] < dataset_stats['Available Cores'] * 2:
        print(f"  • Increase partitions to {dataset_stats['Available Cores'] * 3} for better parallelization")
    
    if dataset_stats['Data Size (MB)'] / dataset_stats['Spark Partitions'] > 200:
        print("  • Consider smaller partitions (target: 128MB per partition) for better load balancing")
    
    print("  • For production: add more executor nodes to distribute computation")
    print("  • For very high dimensions (>1000): consider randomized PCA or sparse PCA")
    print("  • For streaming data: implement incremental PCA with mini-batch updates")

# Run performance analysis
analyze_performance_scalability()

## 10. Advanced PCA Applications and Interpretations

Let's explore practical applications and provide interpretations of our PCA results.

In [ ]:
def analyze_principal_components(pca_results):
    """
    Detailed analysis of principal components and their interpretations
    """
    print("🔍 Principal Component Analysis and Interpretation")
    print("=" * 55)
    
    # Analyze first few components in detail
    n_analyze = min(5, pca_results['n_components'])
    
    for i in range(n_analyze):
        print(f"\n📊 Principal Component {i+1}:")
        print(f"  Explained Variance: {pca_results['explained_variance_ratio'][i]*100:.2f}%")
        print(f"  Cumulative Variance: {pca_results['cumulative_variance_ratio'][i]*100:.2f}%")
        
        # Find top contributing features
        component = pca_results['components'][i]
        
        # Get absolute contributions and sort
        abs_contributions = np.abs(component)
        sorted_indices = np.argsort(abs_contributions)[::-1]
        
        print(f"  Top 5 Contributing Features:")
        for j, idx in enumerate(sorted_indices[:5]):
            feature_name = pca_results['feature_names'][idx]
            contribution = component[idx]
            abs_contribution = abs_contributions[idx]
            sign = '+' if contribution > 0 else '-'
            print(f"    {j+1}. {feature_name[:25]:<25} ({sign}{abs_contribution:.4f})")
    
    # Feature importance analysis
    print(f"\n🎯 Overall Feature Importance Analysis:")
    
    # Calculate feature importance across all components (weighted by explained variance)
    feature_importance = np.zeros(len(pca_results['feature_names']))
    
    for i in range(pca_results['n_components']):
        weight = pca_results['explained_variance_ratio'][i]
        feature_importance += weight * np.abs(pca_results['components'][i])
    
    # Normalize importance scores
    feature_importance = feature_importance / np.sum(feature_importance)
    
    # Get top 10 most important features
    top_feature_indices = np.argsort(feature_importance)[::-1][:10]
    
    print(f"  Top 10 Most Important Features (across all components):")
    for i, idx in enumerate(top_feature_indices):
        feature_name = pca_results['feature_names'][idx]
        importance = feature_importance[idx]
        print(f"    {i+1:2d}. {feature_name[:30]:<30} ({importance*100:.2f}%)")
    
    # Component correlation analysis
    print(f"\n🔗 Component Relationship Analysis:")
    
    if pca_results['n_components'] >= 3:
        # Calculate component correlations (should be near zero for PCA)
        comp_correlations = np.corrcoef(pca_results['components'][:3])
        print(f"  Correlation between first 3 components:")
        for i in range(3):
            for j in range(i+1, 3):
                corr = comp_correlations[i, j]
                print(f"    PC{i+1} ↔ PC{j+1}: {corr:.6f} (should be ~0)")
    
    # Dimensionality reduction recommendations
    print(f"\n📉 Dimensionality Reduction Recommendations:")
    
    variance_thresholds = [0.80, 0.90, 0.95, 0.99]
    for threshold in variance_thresholds:
        n_components_needed = np.argmax(pca_results['cumulative_variance_ratio'] >= threshold) + 1
        reduction_ratio = n_components_needed / len(pca_results['feature_names'])
        print(f"  For {threshold*100:2.0f}% variance: {n_components_needed:3d} components "
              f"({reduction_ratio*100:.1f}% of original dimensions)")
    
    return feature_importance, top_feature_indices

# Perform detailed analysis
feature_importance, top_features = analyze_principal_components(pca_results)

## 11. Practical Applications and Use Cases

Let's explore practical applications of our distributed PCA implementation.

In [ ]:
def demonstrate_pca_applications(pca_results, processed_df, feature_importance):
    """
    Demonstrate practical applications of PCA results
    """
    print("🎯 Practical Applications of Distributed PCA")
    print("=" * 45)
    
    # 1. Data Compression
    print("\n💾 Data Compression Analysis:")
    original_features = len(pca_results['feature_names'])
    
    for variance_threshold in [0.90, 0.95, 0.99]:
        n_components = np.argmax(pca_results['cumulative_variance_ratio'] >= variance_threshold) + 1
        compression_ratio = n_components / original_features
        storage_reduction = (1 - compression_ratio) * 100
        
        print(f"  {variance_threshold*100:.0f}% variance retention:")
        print(f"    Components needed: {n_components}/{original_features}")
        print(f"    Storage reduction: {storage_reduction:.1f}%")
        print(f"    Information loss: {(1-variance_threshold)*100:.1f}%")
    
    # 2. Noise Reduction
    print(f"\n🔇 Noise Reduction Capabilities:")
    
    # Components with low explained variance likely represent noise
    noise_threshold = 0.01  # Components explaining less than 1% variance
    noise_components = np.sum(pca_results['explained_variance_ratio'] < noise_threshold)
    signal_components = pca_results['n_components'] - noise_components
    
    print(f"  Signal components (≥1% variance): {signal_components}")
    print(f"  Noise components (<1% variance): {noise_components}")
    print(f"  Signal-to-noise ratio: {signal_components / max(noise_components, 1):.2f}")
    
    # 3. Feature Selection Insights
    print(f"\n🎯 Feature Selection Insights:")
    
    # Identify redundant features (low importance)
    importance_threshold = np.mean(feature_importance)
    important_features = feature_importance > importance_threshold
    redundant_features = ~important_features
    
    print(f"  Important features (above average): {np.sum(important_features)}")
    print(f"  Potentially redundant features: {np.sum(redundant_features)}")
    print(f"  Feature reduction potential: {np.sum(redundant_features)/len(feature_importance)*100:.1f}%")
    
    # 4. Anomaly Detection Potential
    print(f"\n🚨 Anomaly Detection Applications:")
    
    # In PCA, anomalies often have large reconstruction errors
    # For demonstration, we'll show how this could be implemented
    
    print(f"  Using {pca_results['optimal_components']} components for reconstruction:")
    print(f"    Captures {pca_results['cumulative_variance_ratio'][pca_results['optimal_components']-1]*100:.1f}% of variance")
    print(f"    Reconstruction error threshold could be set based on remaining variance")
    print(f"    Samples with high reconstruction error → potential anomalies")
    
    # 5. Visualization Recommendations
    print(f"\n📊 Data Visualization Recommendations:")
    
    if pca_results['explained_variance_ratio'][0] > 0.5:
        print(f"  • First component explains {pca_results['explained_variance_ratio'][0]*100:.1f}% - good for 1D visualization")
    
    two_comp_variance = pca_results['cumulative_variance_ratio'][1]
    if two_comp_variance > 0.6:
        print(f"  • First 2 components explain {two_comp_variance*100:.1f}% - excellent for 2D scatter plots")
    
    three_comp_variance = pca_results['cumulative_variance_ratio'][2] if pca_results['n_components'] > 2 else 0
    if three_comp_variance > 0.7:
        print(f"  • First 3 components explain {three_comp_variance*100:.1f}% - good for 3D visualization")
    
    # 6. Machine Learning Pipeline Integration
    print(f"\n🤖 ML Pipeline Integration:")
    print(f"  • Use {pca_results['optimal_components']} components as features for downstream ML models")
    print(f"  • Expected speedup in ML training: {original_features / pca_results['optimal_components']:.1f}x")
    print(f"  • Memory reduction in ML models: {(1 - pca_results['optimal_components']/original_features)*100:.1f}%")
    print(f"  • Reduced overfitting risk due to lower dimensionality")
    
    # 7. Real-world Scenario Applications
    print(f"\n🌍 Real-world Application Scenarios:")
    print(f"  📈 Financial Data: Reduce 100s of market indicators to key factors")
    print(f"  🏥 Healthcare: Compress high-dimensional patient data for analysis")
    print(f"  🛒 E-commerce: Reduce customer behavior features for recommendation systems")
    print(f"  📱 IoT Sensors: Compress multi-sensor time series data")
    print(f"  🔬 Genomics: Reduce thousands of gene expression features")
    print(f"  📸 Computer Vision: Preprocess high-dimensional image features")

# Demonstrate applications
demonstrate_pca_applications(pca_results, processed_df, feature_importance)

## 12. Conclusions and Future Improvements

Let's summarize our findings and discuss potential improvements.

In [ ]:
def generate_conclusions_and_recommendations():
    """
    Generate comprehensive conclusions and recommendations
    """
    print("📝 Project Conclusions and Recommendations")
    print("=" * 50)
    
    # Key achievements
    print("\n🏆 Key Achievements:")
    print(f"  ✅ Successfully implemented distributed PCA using Apache Spark")
    print(f"  ✅ Processed {processed_df.count():,} samples with {len(feature_names)} features")
    print(f"  ✅ Achieved {pca_results['cumulative_variance_ratio'][pca_results['optimal_components']-1]*100:.1f}% variance capture with {pca_results['optimal_components']} components")
    print(f"  ✅ Validated results against scikit-learn with high accuracy")
    print(f"  ✅ Demonstrated scalability for big data scenarios")
    
    # Technical insights
    print(f"\n🔬 Technical Insights:")
    print(f"  📊 First component captured {pca_results['explained_variance_ratio'][0]*100:.1f}% of total variance")
    print(f"  📊 Achieved {(1 - pca_results['optimal_components']/len(feature_names))*100:.1f}% dimensionality reduction")
    print(f"  📊 SVD computation time: {pca_results['svd_time']:.2f}s for {len(feature_names)} dimensions")
    print(f"  📊 Memory-efficient processing through Spark's distributed architecture")
    
    # Architecture benefits
    print(f"\n🏗️ Architecture Benefits Demonstrated:")
    print(f"  🚀 Horizontal scalability through data partitioning")
    print(f"  💾 Memory efficiency via distributed computation")
    print(f"  ⚡ Parallel processing across multiple cores")
    print(f"  🔄 Fault tolerance through Spark's RDD lineage")
    
    # Challenges addressed
    print(f"\n💪 Challenges Successfully Addressed:")
    print(f"  ⚖️ Memory constraints: Distributed matrix operations")
    print(f"  📈 Computational complexity: Parallel SVD computation")
    print(f"  🔀 Data shuffling: Optimized partitioning strategy")
    print(f"  🎯 High dimensionality: Efficient feature selection and scaling")
    
    # Future improvements
    print(f"\n🚀 Future Improvements and Extensions:")
    
    print(f"  🔧 Technical Enhancements:")
    print(f"    • Implement Incremental PCA for streaming data")
    print(f"    • Add Randomized PCA for very high-dimensional data (>10K features)")
    print(f"    • Implement Sparse PCA for datasets with many zero values")
    print(f"    • Add Kernel PCA for non-linear dimensionality reduction")
    
    print(f"  ⚡ Performance Optimizations:")
    print(f"    • Implement approximate SVD algorithms for faster computation")
    print(f"    • Add GPU acceleration for matrix operations")
    print(f"    • Optimize data serialization and caching strategies")
    print(f"    • Implement adaptive partitioning based on data characteristics")
    
    print(f"  🔍 Advanced Analytics:")
    print(f"    • Add automated feature importance ranking")
    print(f"    • Implement PCA-based anomaly detection")
    print(f"    • Add component interpretation and visualization tools")
    print(f"    • Integrate with MLlib pipelines for end-to-end ML workflows")
    
    print(f"  🌐 Production Considerations:")
    print(f"    • Add model serialization and deployment capabilities")
    print(f"    • Implement real-time PCA updates for streaming data")
    print(f"    • Add comprehensive monitoring and logging")
    print(f"    • Create RESTful API for PCA-as-a-Service")
    
    # Best practices learned
    print(f"\n📚 Best Practices Learned:")
    print(f"  1. Always scale features before PCA to avoid bias toward high-variance features")
    print(f"  2. Choose optimal number of partitions based on data size and cluster resources")
    print(f"  3. Cache intermediate results for operations that will be reused")
    print(f"  4. Monitor memory usage and adjust Spark configuration accordingly")
    print(f"  5. Validate distributed results against known implementations")
    print(f"  6. Consider computational complexity when choosing number of components")
    
    # Final recommendations
    print(f"\n🎯 Final Recommendations:")
    print(f"  For datasets with:")
    print(f"    • <100K samples: Use scikit-learn PCA (faster setup)")
    print(f"    • 100K-10M samples: Use Spark PCA (demonstrated approach)")
    print(f"    • >10M samples: Use Spark with incremental/randomized PCA")
    print(f"    • >1000 features: Consider feature pre-selection or sparse methods")
    print(f"    • Streaming data: Implement online/incremental PCA")
    
    print(f"\n✨ This implementation provides a solid foundation for big data PCA")
    print(f"   and can be extended for various production scenarios!")

# Generate final conclusions
generate_conclusions_and_recommendations()

## 13. Cleanup and Resource Management

In [ ]:
# Clean up Spark resources
print("🧹 Cleaning up Spark resources...")

# Unpersist cached DataFrames
try:
    spark_df.unpersist()
    processed_df.unpersist()
    print("✅ Cached DataFrames unpersisted")
except:
    pass

# Stop Spark session
spark.stop()
print("✅ Spark session stopped")

print("\n🎉 Big Data PCA Analysis Complete!")
print("📋 Summary of deliverables:")
print("  • Distributed PCA implementation using Apache Spark")
print("  • Performance analysis and scalability assessment")
print("  • Validation against standard library implementations")
print("  • Comprehensive visualizations and interpretations")
print("  • Practical applications and use case demonstrations")
print("  • Future improvements and production recommendations")